In [0]:
import sparknlp
print("Spark NLP version")
sparknlp.version()
print("Apache Spark version")
spark.version

Spark NLP version
Apache Spark version
Out[2]: '3.1.0'

In [0]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
from sparknlp.common import *

In [0]:
#/FileStore/tables/news_category_train.csv
#/FileStore/tables/news_category_test.csv

In [0]:
spark=sparknlp.start()

In [0]:
train_df=spark.read.option('header','true').csv('dbfs:/FileStore/tables/news_category_train.csv')
train_df.show(5)

+--------+--------------------+
category| description|
+--------+--------------------+
Business| Short sellers, W...|
Business| Private investme...|
Business| Soaring crude pr...|
Business| Authorities have...|
Business| Tearaway world o...|
+--------+--------------------+
only showing top 5 rows

In [0]:
train_df.count()

Out[6]: 120000

In [0]:
from pyspark.sql import functions as func

In [0]:
train_df.groupBy('category').count().show()

+--------+-----+
category|count|
+--------+-----+
 World|30000|
Sci/Tech|30000|
 Sports|30000|
Business|30000|
+--------+-----+

In [0]:
test_df=spark.read.option('header','true').csv('dbfs:/FileStore/tables/news_category_test.csv')
test_df.show(5)

+--------+--------------------+
category| description|
+--------+--------------------+
Business|Unions representi...|
Sci/Tech| TORONTO, Canada ...|
Sci/Tech| A company founde...|
Sci/Tech| It's barely dawn...|
Sci/Tech| Southern Califor...|
+--------+--------------------+
only showing top 5 rows

In [0]:
test_df.groupBy('category').count().show()

+--------+-----+
category|count|
+--------+-----+
 World| 1900|
Sci/Tech| 1900|
 Sports| 1900|
Business| 1900|
+--------+-----+

In [0]:
training_data,test_data=train_df.randomSplit([0.7,0.3],seed=101)

In [0]:
training_data.count()

Out[12]: 83966

In [0]:
test_data.count()

Out[13]: 36034

In [0]:
document_assembler=DocumentAssembler().setInputCol("description").setOutputCol("document")

In [0]:
tokenizer=Tokenizer().setInputCols("document").setOutputCol("token")

In [0]:
normalizer=Normalizer().setInputCols("token").setOutputCol("normalized")

In [0]:
stopwords_cleaner=StopWordsCleaner().setInputCols("normalized").setOutputCol("cleanToken")

In [0]:
lemma=LemmatizerModel.pretrained().setInputCols("cleanToken").setOutputCol("lemma")

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][ / ][ — ][OK!]

In [0]:
embeddings=WordEmbeddingsModel().pretrained().setInputCols(['document','lemma']).setOutputCol("embeddings")

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
embeddingsSentence=SentenceEmbeddings().setInputCols(['document','embeddings']).setOutputCol('sentence_embeddings')

In [0]:
classifier_dl=ClassifierDLApproach().setInputCols("sentence_embeddings").setOutputCol("class").setLabelColumn("category").setMaxEpochs(1).setEnableOutputLogs(True)

In [0]:
pipeline=Pipeline(stages=[document_assembler,tokenizer,normalizer,stopwords_cleaner,lemma,embeddings,embeddingsSentence,classifier_dl])

In [0]:
model=pipeline.fit(training_data)

In [0]:
preds=model.transform(test_data)

In [0]:
preds.show(5)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
category| description| document| token| normalized| cleanToken| lemma| embeddings| sentence_embeddings| class|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Business| A federal judge...|[{document, 0, 14...|[{token, 2, 2, A,...|[{token, 2, 2, A,...|[{token, 4, 10, f...|[{token, 4, 10, f...|[{word_embeddings...|[{sentence_embedd...|[{category, 0, 14...|
Business| A group led by ...|[{document, 0, 18...|[{token, 2, 2, A,...|[{token, 2, 2, A,...|[{token, 4, 8, gr...|[{token, 4, 8, gr...|[{word_embeddings...|[{sentence_embedd...|[{category, 0, 18...|
Business| Although publis...|[{document, 0, 24...|[{token, 2, 9, Al...|[{token, 2, 9, Al...|[{token, 2, 9, Al...|[{token, 2, 9, Al...|[{word_embeddings...|[{sentence_embedd...|[{category, 0, 24...|
Business| Americans paid ...|[{document, 0, 19...|[{token, 2, 10, A...|[{token, 2, 10, A...|[{token, 2, 10, A...|[{token, 2, 10, A...|[{word_embeddings...|[{sentence_embedd...|[{category, 0, 19...|
Business| Applied Materia...|[{document, 0, 23...|[{token, 2, 8, Ap...|[{token, 2, 8, Ap...|[{token, 2, 8, Ap...|[{token, 2, 8, Ap...|[{word_embeddings...|[{sentence_embedd...|[{category, 0, 23...|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [0]:
preds.select('category','description',"class.result").show(5)

+--------+--------------------+----------+
category| description| result|
+--------+--------------------+----------+
Business| A federal judge...|[Business]|
Business| A group led by ...|[Business]|
Business| Although publis...|[Business]|
Business| Americans paid ...|[Business]|
Business| Applied Materia...|[Business]|
+--------+--------------------+----------+
only showing top 5 rows

In [0]:
pred_df=preds.select('category','description',"class.result").toPandas()

In [0]:
pred_df.head()

Out[29]:

,category,description,result
0,Business,A federal judge has scheduled jury selection...,[Business]
1,Business,A group led by privately held Colony Capital...,[Business]
2,Business,Although published reports yesterday claimed...,[Business]
3,Business,Americans paid their credit card bills on ti...,[Business]
4,Business,Applied Materials beat Wall Street #39;s thi...,[Business]


In [0]:
pred_df["result"]=pred_df["result"].apply(lambda x:x[0])
pred_df["result"].head(5)

Out[30]: 0 Business
1 Business
2 Business
3 Business
4 Business
Name: result, dtype: object

In [0]:
from sklearn.metrics import classification_report

In [0]:
print(classification_report(pred_df["result"],pred_df["category"]))

precision recall f1-score support

 Business 0.86 0.81 0.83 9637
 Sci/Tech 0.80 0.86 0.83 8527
 Sports 0.96 0.94 0.95 9083
 World 0.87 0.88 0.88 8787

 accuracy 0.87 36034
 macro avg 0.87 0.87 0.87 36034
weighted avg 0.87 0.87 0.87 36034

In [0]:
print(classification_report(pred_df["category"],pred_df["result"]))

precision recall f1-score support

 Business 0.81 0.86 0.83 9140
 Sci/Tech 0.86 0.80 0.83 9189
 Sports 0.94 0.96 0.95 8843
 World 0.88 0.87 0.88 8862

 accuracy 0.87 36034
 macro avg 0.87 0.87 0.87 36034
weighted avg 0.87 0.87 0.87 36034

In [0]:
document = DocumentAssembler().setInputCol("description").setOutputCol("document")
use = UniversalSentenceEncoder.pretrained().setInputCols(["document"]).setOutputCol("sentence_embeddings")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
classsifierdl = ClassifierDLApproach().setInputCols(["sentence_embeddings"]).setOutputCol("class").setLabelColumn("category").setMaxEpochs(1)

In [0]:
pipeline = Pipeline(stages=[document,use,classsifierdl])

In [0]:
new_model=pipeline.fit(training_data)

In [0]:
new_predict=new_model.transform(test_data)

In [0]:
new_predict.select('category','description',"class.result").show(10)

+--------+--------------------+----------+
category| description| result|
+--------+--------------------+----------+
Business| A federal judge...|[Business]|
Business| A group led by ...|[Business]|
Business| Although publis...|[Business]|
Business| Americans paid ...|[Business]|
Business| Applied Materia...|[Business]|
Business| Asian stocks cl...|[Business]|
Business| BHP Billiton, t...|[Business]|
Business| BHP Billiton, t...|[Business]|
Business| Cantor Fitzgera...|[Business]|
Business| Citigroup Inc.,...|[Business]|
+--------+--------------------+----------+
only showing top 10 rows

In [0]:
new_pred=new_predict.select('category','description',"class.result").toPandas()

In [0]:
new_pred.head(10)

Out[41]:

,category,description,result
0,Business,A federal judge has scheduled jury selection...,[Business]
1,Business,A group led by privately held Colony Capital...,[Business]
2,Business,Although published reports yesterday claimed...,[Business]
3,Business,Americans paid their credit card bills on ti...,[Business]
4,Business,Applied Materials beat Wall Street #39;s thi...,[Business]
5,Business,"Asian stocks closed mainly higher Tuesday, l...",[Business]
6,Business,"BHP Billiton, the world #39;s biggest miner,...",[Business]
7,Business,"BHP Billiton, the world #39;s biggest mining...",[Business]
8,Business,"Cantor Fitzgerald LP, one of the two largest...",[Business]
9,Business,"Citigroup Inc., the world #39;s biggest bank...",[Business]


In [0]:
new_pred["result"]=new_pred["result"].apply(lambda x:x[0])
new_pred["result"].head(10)

Out[42]: 0 Business
1 Business
2 Business
3 Business
4 Business
5 Business
6 Business
7 Business
8 Business
9 Business
Name: result, dtype: object

In [0]:
print(classification_report(new_pred["result"],new_pred["category"]))

precision recall f1-score support

 Business 0.83 0.84 0.84 9020
 Sci/Tech 0.88 0.83 0.85 9729
 Sports 0.98 0.94 0.96 9280
 World 0.85 0.94 0.89 8005

 accuracy 0.88 36034
 macro avg 0.88 0.89 0.88 36034
weighted avg 0.89 0.88 0.88 36034

In [0]:
print(classification_report(new_pred["category"],new_pred["result"]))

precision recall f1-score support

 Business 0.84 0.83 0.84 9140
 Sci/Tech 0.83 0.88 0.85 9189
 Sports 0.94 0.98 0.96 8843
 World 0.94 0.85 0.89 8862

 accuracy 0.88 36034
 macro avg 0.89 0.88 0.88 36034
weighted avg 0.88 0.88 0.88 36034

In [0]:
from sparknlp.base import LightPipeline

In [0]:
light_model=LightPipeline(new_model)

In [0]:
test_data.select("description").take(2)

Out[47]: [Row(description=' A federal judge has scheduled jury selection in the fraud trial of fired HealthSouth Corp. chief executive Richard Scrushy to begin Jan. 5.'),
 Row(description=' A group led by privately held Colony Capital is close to reaching an agreement to buy the Atlantic City Hilton and three other casinos in the Chicago area and Mississippi for about $1.')]

In [0]:
from pyspark.sql.types import StringType

In [0]:
dfTest = spark.createDataFrame([
    "Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.",
    "Scientists have discovered irregular lumps beneath the icy surface of Jupiter's largest moon, Ganymede. These irregular masses may be rock formations, supported by Ganymede's icy shell for billions of years..."
], StringType()).toDF("description")

In [0]:
dfTest.show()

+--------------------+
 description|
+--------------------+
Unions representi...|
Scientists have d...|
+--------------------+

In [0]:
light_prediction = light_model.transform(dfTest)

In [0]:
print(light_prediction)

DataFrame[description: string, document: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, sentence_embeddings: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, class: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>]

In [0]:
light_prediction.select("class.result","description").show()

+----------+--------------------+
 result| description|
+----------+--------------------+
[Business]|Unions representi...|
[Sci/Tech]|Scientists have d...|
+----------+--------------------+

In [0]:
light_df=light_prediction.select("class.result","description").toPandas()
light_df

Out[54]:

,result,description
0,[Business],Unions representing workers at Turner Newall s...
1,[Sci/Tech],Scientists have discovered irregular lumps ben...


In [0]:
light_prediction.select("class.metadata").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------+
metadata |
+-----------------------------------------------------------------------------------------------------------------+
[{Sports -> 2.1335938E-5, Business -> 0.9998834, World -> 7.3573654E-5, Sci/Tech -> 2.1628119E-5, sentence -> 0}]|
[{Sports -> 3.4068994E-9, Business -> 1.8231168E-8, World -> 1.5277089E-11, Sci/Tech -> 1.0, sentence -> 0}] |
+-----------------------------------------------------------------------------------------------------------------+

In [0]:
light_preds=light_model.transform(test_df)

In [0]:
light_preds.select('category','description',"class.result").show(20)

+--------+--------------------+----------+
category| description| result|
+--------+--------------------+----------+
Business|Unions representi...|[Business]|
Sci/Tech| TORONTO, Canada ...|[Sci/Tech]|
Sci/Tech| A company founde...|[Sci/Tech]|
Sci/Tech| It's barely dawn...|[Sci/Tech]|
Sci/Tech| Southern Califor...|[Business]|
Sci/Tech|"The British Depa...|[Sci/Tech]|
Sci/Tech|"confessed author...|[Sci/Tech]|
Sci/Tech|\\FOAF/LOAF and ...|[Sci/Tech]|
Sci/Tech|"Wiltshire Police...|[Sci/Tech]|
Sci/Tech|In its first two ...|[Sci/Tech]|
Sci/Tech| A group of techn...|[Sci/Tech]|
Sci/Tech| Apple Computer I...|[Sci/Tech]|
Sci/Tech| Free Record Shop...|[Sci/Tech]|
Sci/Tech|A giant 100km col...|[Sci/Tech]|
Sci/Tech|"Dolphin groups, ...|[Sci/Tech]|
Sci/Tech|Tyrannosaurus rex...|[Sci/Tech]|
Sci/Tech| Scientists have...|[Sci/Tech]|
Sci/Tech| ESAs Mars Expre...|[Sci/Tech]|
Sci/Tech|When did life beg...|[Sci/Tech]|
Sci/Tech|update Earnings p...|[Business]|
+--------+--------------------+----------+
only showing top 20 rows

In [0]:
new_df=light_preds.select('category','description',"class.result").toPandas()

In [0]:
new_df.head(10)

Out[59]:

,category,description,result
0,Business,Unions representing workers at Turner Newall...,[Business]
1,Sci/Tech,"TORONTO, Canada A second team of rocketeer...",[Sci/Tech]
2,Sci/Tech,A company founded by a chemistry researcher a...,[Sci/Tech]
3,Sci/Tech,It's barely dawn when Mike Fitzpatrick starts...,[Sci/Tech]
4,Sci/Tech,Southern California's smog fighting agency we...,[Business]
5,Sci/Tech,"""The British Department for Education and Skil...",[Sci/Tech]
6,Sci/Tech,"""confessed author of the Netsky and Sasser vir...",[Sci/Tech]
7,Sci/Tech,\\FOAF/LOAF and bloom filters have a lot of i...,[Sci/Tech]
8,Sci/Tech,"""Wiltshire Police warns about """"phishing"""" aft...",[Sci/Tech]
9,Sci/Tech,"In its first two years, the UK's dedicated car...",[Sci/Tech]


In [0]:
new_df['result'] = new_df['result'].apply(lambda x : x[0])

In [0]:
new_df['result'].head()

Out[62]: 0 Business
1 Sci/Tech
2 Sci/Tech
3 Sci/Tech
4 Business
Name: result, dtype: object

In [0]:
print(classification_report(new_df['result'],new_df['category']))

precision recall f1-score support

 Business 0.82 0.84 0.83 1846
 Sci/Tech 0.88 0.82 0.85 2043
 Sports 0.98 0.93 0.96 1997
 World 0.84 0.93 0.88 1714

 accuracy 0.88 7600
 macro avg 0.88 0.88 0.88 7600
weighted avg 0.88 0.88 0.88 7600